# 数据转换

首先导入pandas相关模块。

In [1]:
import pandas as pd
import numpy as np

## 删除重复项

`duplicated`方法返回一个boolean类型的`Series`对象，原对象中出现的重复项，除第一次出现的以外，均为True，其余项为False。

`drop_duplicates`方法返回`duplicated`方法中所有为False的项。`drop_duplicates`可接受一个列表类型参数，指定需要判重的列名。

In [4]:
data = pd.Series([1, 1, 2, 1, 3, 4, 5])
data.duplicated()

0    False
1     True
2    False
3     True
4    False
5    False
6    False
dtype: bool

In [6]:
data.drop_duplicates()

0    1
2    2
4    3
5    4
6    5
dtype: int64

In [7]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [8]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [9]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [11]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [12]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [13]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## 使用函数或映射转换数据



In [14]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


对于`Series`对象，`map`方法接受一个`dict`对象作为参数，会依次将`Sereis`对象中的各元素通过`dict`对象映射为对应的新值。

In [15]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [17]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [18]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

## 值的替换

`replace`方法可以将`Series`或`DataFrame`对象中相应的值替换为新值。

In [22]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [23]:
# 将-999替换为nan
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [24]:
# 将-999与-1000替换为nan
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [25]:
# 将-999替换为nan，将-1000替换为0
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [26]:
# 将-999替换为nan，将-1000替换为0
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [29]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data.replace({'one': "I", 'two': 'II'})

,k1,k2
0,I,1
1,II,1
2,I,2
3,II,3
4,I,3
5,II,4
6,II,4


## 重命名行列索引名

`map`方法对行列索引名仍然有效。

`rename`方法可以在不修改原对象行列索引名的情况下修改行列索引名并返回一个新对象，`inplace`属性可以指定对原对象修改。

In [30]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [31]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [32]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [35]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [36]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [37]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## 数据离散化与分箱

`pd.cut(data, bins)`函数实现了将数据`data`以`bins`为标准进行分箱的功能。该函数返回一个`Categorical`对象，可以被视为一个包含各个bin名的字符串数组。

In [38]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [39]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [41]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [44]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [45]:
# right为False指定bins为左闭右开区间
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [47]:
# 通过label为每一个bins命名
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

为`bins`属性传递一个整数而非列表，则将根据传入数据的最大值与最小值分配等长bins个区间。

In [49]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.46, 0.69], (0.014, 0.24], (0.69, 0.91], (0.014, 0.24], (0.69, 0.91], ..., (0.014, 0.24], (0.24, 0.46], (0.46, 0.69], (0.46, 0.69], (0.69, 0.91]]
Length: 20
Categories (4, interval[float64]): [(0.014, 0.24] < (0.24, 0.46] < (0.46, 0.69] < (0.69, 0.91]]

`qcut`函数与`cut`函数类似，其根据分位数进行分箱。

In [50]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
cats

[(0.667, 3.152], (-2.665, -0.661], (0.667, 3.152], (0.667, 3.152], (-0.661, 0.0565], ..., (0.0565, 0.667], (-2.665, -0.661], (0.667, 3.152], (0.0565, 0.667], (0.667, 3.152]]
Length: 1000
Categories (4, interval[float64]): [(-2.665, -0.661] < (-0.661, 0.0565] < (0.0565, 0.667] < (0.667, 3.152]]

In [51]:
pd.value_counts(cats)

(0.667, 3.152]      250
(0.0565, 0.667]     250
(-0.661, 0.0565]    250
(-2.665, -0.661]    250
dtype: int64

In [52]:
# 指定各个分位数
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(0.0565, 1.297], (-2.665, -1.169], (1.297, 3.152], (0.0565, 1.297], (-1.169, 0.0565], ..., (0.0565, 1.297], (-2.665, -1.169], (1.297, 3.152], (0.0565, 1.297], (1.297, 3.152]]
Length: 1000
Categories (4, interval[float64]): [(-2.665, -1.169] < (-1.169, 0.0565] < (0.0565, 1.297] < (1.297, 3.152]]

## 查询与过滤极值

In [53]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.019183,0.008867,-0.041299,0.010426
std,0.980973,0.971111,0.967567,1.003373
min,-3.009662,-3.346494,-3.460126,-3.284157
25%,-0.636384,-0.631603,-0.708976,-0.702936
50%,-0.000510,0.024380,-0.056037,0.022297
75%,0.669695,0.676302,0.587730,0.689891
max,3.256187,3.089172,2.933142,3.618248


In [54]:
# 查询DataFrame的第二列中绝对值大于3的元素
col = data[2]
col[np.abs(col) > 3]

431   -3.460126
Name: 2, dtype: float64

In [55]:
# 返回DataFrame中满足包含绝对值大于3的元素的所有行
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
8,3.256187,-0.481106,0.706392,1.465579
41,-0.727213,0.048973,-0.204942,3.056131
68,1.321946,-3.346494,1.741245,-0.443016
431,-1.393152,-0.453846,-3.460126,1.313372
439,1.149094,3.089172,-0.219524,-1.980719
443,0.510946,-0.412936,1.148862,3.618248
516,0.809355,-2.186855,0.200397,-3.284157
588,3.070346,-0.243525,-0.348210,0.653305
639,-1.016170,-0.467006,-0.977177,3.454961
938,3.145417,0.163769,0.167833,2.130222


In [56]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.018721,0.009124,-0.040839,0.009581
std,0.979460,0.969695,0.966048,0.998805
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.636384,-0.631603,-0.708976,-0.702936
50%,-0.000510,0.024380,-0.056037,0.022297
75%,0.669695,0.676302,0.587730,0.689891
max,3.000000,3.000000,2.933142,3.000000


In [57]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,1.0,1.0,1.0,-1.0
2,1.0,-1.0,-1.0,-1.0
3,1.0,-1.0,1.0,-1.0
4,1.0,1.0,-1.0,1.0


## 排列与随机取样

`numpy.random.permutation`函数返回`DataFrame`所有行与列的一个随机排列。

`take`方法与`iloc`方法类似。

`sample`方法随机选取`Series`或`DataFrame`对象的一个子集，`inplace`参数在原对象上取样后返回。

In [58]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([0, 4, 1, 2, 3])

In [59]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [60]:
df.take(sampler)

,0,1,2,3
0,0,1,2,3
4,16,17,18,19
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [62]:
df.iloc[sampler]

,0,1,2,3
0,0,1,2,3
4,16,17,18,19
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [63]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
0,0,1,2,3
1,4,5,6,7


In [64]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

3    6
1    7
1    7
1    7
1    7
4    4
4    4
0    5
4    4
4    4
dtype: int64

## indicator/dummy变量

In [65]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [66]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [67]:
# prefix属性为indicator对象的列名指定前缀
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


// TODO